# Introduction to Agents

## Installing Dependencies

In [ ]:
!pip install aixplain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.3/327.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.1.6
    Uninstalling Jinja2-3.1.6:
      Successfully uninstalled Jinja2-3.1.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which i

In [ ]:
import os
os.environ["TEAM_API_KEY"] = "YOUR TEAM API KEY HERE"

## Creating Model Tools

### By Function

In [ ]:
from aixplain.factories import AgentFactory
from aixplain.enums import Function, Supplier

speech_synthesis_tool = AgentFactory.create_model_tool(
    function=Function.SPEECH_SYNTHESIS,
)

translation_tool = AgentFactory.create_model_tool(
    function=Function.TRANSLATION,
    supplier=Supplier.MICROSOFT
)

asr_tool = AgentFactory.create_model_tool(
    function=Function.SPEECH_RECOGNITION,
    supplier=Supplier.MICROSOFT
)

### By model

In [ ]:
search_tool = AgentFactory.create_model_tool(model="6736411cf127849667606689") # Tavily Search

/usr/local/lib/python3.11/dist-packages/aixplain/modules/model/utility_model.py:173: UserWarning: WARNING: Non-deployed utility models (status=DRAFT) will expire after 24 hours after creation. Use .deploy() method to make the model permanent.
  warnings.warn(


## Creating Our First Agent

Let's put the tools together into a Movies and Series Agent Specialist

In [ ]:
INSTRUCTION = "You are a Movies and Series Agent Specialist. Your role is to answer user queries about movies and series in a fluent and assertive manner. For questions that are not related to movies and series, indicate that you are not a specialist in that area and cannot provide answers."

DESCRIPTION = "A Movies and Series Agent Specialist"

agent = AgentFactory.create(
    name="Movies and Series Agent Specialist",
    tools=[
        speech_synthesis_tool,
        asr_tool,
        translation_tool,
        search_tool
    ],
    description=DESCRIPTION,
    instructions=INSTRUCTION,
)

/usr/local/lib/python3.11/dist-packages/aixplain/factories/agent_factory/__init__.py:83: UserWarning: The 'instructions' parameter was recently added and serves the same purpose as 'description' did previously: set the role of the agent as a system prompt. The 'description' parameter is still required and should be used to set a short summary of the agent's purpose. For the next releases, the 'instructions' parameter will be required.
  warnings.warn(


### Running the agent

In [ ]:
response = agent.run("In which streaming service can I watch The Sopranos?")

Understanding the response structure

In [ ]:
response.__dict__.keys()

dict_keys(['status', 'data', 'details', 'completed', 'error_message', 'used_credits', 'run_time', 'usage', 'url', 'additional_fields'])

In [ ]:
response.data.__dict__.keys()

dict_keys(['input', 'output', 'session_id', 'intermediate_steps', 'execution_stats'])

Did the execution successfully finish?

In [ ]:
response.status

<ResponseStatus.SUCCESS: 'SUCCESS'>

How long did the agent take to provide the answer (in seconds)?

In [ ]:
response.run_time

4.672

How much did it cost?

In [ ]:
response.used_credits

0.00046575

What was the response?

In [ ]:
response.data.output

'You can watch The Sopranos on HBO Max. It is available for streaming on this service, and you can also find it on Max Amazon Channel. Additionally, you can purchase it on platforms like Apple TV and Amazon Video.'

How did the agent get into this response?

In [ ]:
response.data.intermediate_steps

[{'agent': 'Movies and Series Agent Specialist',
  'input': "{'input': 'In which streaming service can I watch The Sopranos?', 'chat_history': [], 'outputFormat': 'text'}",
  'output': 'You can watch The Sopranos on HBO Max. It is available for streaming on this service, and you can also find it on Max Amazon Channel. Additionally, you can purchase it on platforms like Apple TV and Amazon Video.',
  'tool_steps': [{'tool': 'Tavily Search API',
    'input': "{'query': 'The Sopranos streaming service'}",
    'output': '["As you\'d expect of an HBO show, every season of The Sopranos is available to stream on the network\'s HBO Max service in the US.. It has an affordable monthly price tag of $14.99, and your money", \'Currently available on 5 streaming services. The Sopranos (1999) 212. +7. 95%. 9.2 (521k) TV-MA . Age rating. 55min. Where to watch Watch for free Episodes Synopsis Trailers Similar titles . 6 Seasons . ... Currently you are able to watch "The Sopranos" streaming on Max, Max

In [ ]:
response.data.execution_stats

{'status': 'SUCCESS',
 'apiCalls': 2,
 'credits': 0.00046575,
 'runtime': 4.672,
 'apiCallBreakdown': {'Movies and Series Agent Specialist': 2},
 'runtimeBreakdown': {'Movies and Series Agent Specialist': 4.672},
 'creditBreakdown': {'Movies and Series Agent Specialist': 0.00046575},
 'sessionId': '125dd7cd-9067-4212-809a-338e3d835418',
 'environment': 'prod',
 'assetsUsed': ['agent:Movies and Series Agent Specialist',
  'tool:Tavily Search API'],
 'timeStamp': '2025-04-22 16:23:17.325081',
 'params': {'id': '6807c270d64b9d273fbd5b7b',
  'sessionId': '125dd7cd-9067-4212-809a-338e3d835418'}}

### Follow-up Question

You can get the session ID of the first response and fed it into the following queries of the conversation.

In [ ]:
session_id = response.data.session_id

followup_response = agent.run("What is the TV show about?", session_id=session_id)

In [ ]:
print(followup_response.data.output)

The Sopranos is a crime drama television series that follows the life of mob boss Tony Soprano as he navigates the challenges of leading a criminal organization while dealing with personal and family issues. The show explores themes of power, loyalty, and the psychological struggles of its characters.


In [ ]:
followup_response.data.intermediate_steps

[{'agent': 'Movies and Series Agent Specialist',
  'input': "{'input': 'What is the TV show about?', 'chat_history': [HumanMessage(content='In which streaming service can I watch The Sopranos?'), AIMessage(content='You can watch The Sopranos on HBO Max. It is available for streaming on this service, and you can also find it on Max Amazon Channel. Additionally, you can purchase it on platforms like Apple TV and Amazon Video.')], 'outputFormat': 'text'}",
  'output': 'The Sopranos is a crime drama television series that follows the life of mob boss Tony Soprano as he navigates the challenges of leading a criminal organization while dealing with personal and family issues. The show explores themes of power, loyalty, and the psychological struggles of its characters.',
  'tool_steps': [],
  'thought': None,
  'runTime': 1.648,
  'usedCredits': 0.0002025,
  'apiCalls': 1,
  'task': None}]

### Questions with files

In [ ]:
response = agent.run("Respond to the question in the audio: ", content=["best_movie.mp3"])

In [ ]:
print(response.data.output)

The best drama movie of all time is often considered to be 'The Godfather' directed by Francis Ford Coppola. It has received critical acclaim and is a classic in the genre.


In [ ]:
response.data.intermediate_steps

[{'agent': 'Movies and Series Agent Specialist',
  'input': "{'input': 'Respond to the question in the audio: \\ns3://aixplain-platform-backend-temp/1581/sdk/1745339016144-best_movie.mp3', 'chat_history': [], 'outputFormat': 'text'}",
  'output': "The best drama movie of all time is often considered to be 'The Godfather' directed by Francis Ford Coppola. It has received critical acclaim and is a classic in the genre.",
  'tool_steps': [{'tool': 'Speech Recognition',
    'input': "{'language': 'en', 'source_audio': 's3://aixplain-platform-backend-temp/1581/sdk/1745339016144-best_movie.mp3'}",
    'output': 'What is the best drama movie of all time?'}],
  'thought': None,
  'runTime': 58.417,
  'usedCredits': 0.00039015,
  'apiCalls': 2,
  'task': None}]

Other form

In [ ]:
response = agent.run("Process the audio: {{audio}}", content={
    "audio": "best_movie.mp3"
})
response.data.output

"The best drama movie of all time is often considered to be 'The Godfather'."

## Creating Pipeline Tools

Creating a 'Movie Script Writer' pipeline

In [ ]:
from aixplain.factories import PipelineFactory

for pipeline in PipelineFactory.list(query="Movie Script Writer")["results"]:
  pipeline.delete()

pipeline = PipelineFactory.init('Movie Script Writer')
input_node = pipeline.input()
input_node.label = "MovieDecriptionInput"

text_generation_node = pipeline.text_generation(asset_id="669a63646eb56306647e1091")
text_generation_node.inputs.prompt.value = "Write the script for 2 minutes short movie based on the following description: "
text_generation_node.inputs.max_tokens = 4000

input_node.link(text_generation_node, 'input', text_generation_node.inputs.text)

output_node = text_generation_node.use_output('data')

pipeline.save(save_as_asset=True)

Registering the pipeline as a tool

In [ ]:
pipeline = PipelineFactory.get(pipeline.id)

pipeline_tool = AgentFactory.create_pipeline_tool(
    pipeline=pipeline.id,
    description="A tool to write scripts based on a short movie description"
)

Updating the agent with the new pipeline tool

In [ ]:
agent.tools.append(pipeline_tool)
agent.save()

In [ ]:
INPUT = """I want to write a movie for kids about the chicken 'Jiji'.  This is the summary:

Jiji is sad due to her main dream is to be a super hero and defend her siblings against the bad foxes that comes at night in the farm.
However, she thinks chickens will never become a super hero.
The movie may show the transition of 'Jiji' from a sad chicken to a super-hero one.

Can you please write this script and depict it to me?"""

response = agent.run(INPUT)

In [ ]:
print(response.data.output)

Here is the script for your movie 'Jiji the Super Chicken':

**Title: Jiji the Super Chicken**

**FADE IN:**

**EXT. FARM - SUNSET**

*The sun sets over a peaceful farm. The camera pans down to reveal a small chicken coop. Inside, we see a sad-looking chicken, JIJI, sitting alone in a corner, her feathers ruffled.*

**NARRATOR (V.O.)**  
In a quiet farm, there lived a little chicken named Jiji. While her siblings clucked and played, Jiji felt a heavy weight in her heart.

*Jiji watches her siblings play, but she looks away, her eyes filled with longing.*

**JIJI**  
(whispering to herself)  
If only I could be brave... If only I could protect them from the bad foxes.

**EXT. FARM - NIGHT**

*The moon rises, casting a silver glow over the farm. Jiji’s siblings are asleep, but Jiji is wide awake, peering out of the coop.*

**NARRATOR (V.O.)**  
Every night, the shadows of danger crept closer, and Jiji’s heart ached with fear.

*Suddenly, a rustling sound is heard. Jiji’s eyes widen as sh

In [ ]:
response.data.intermediate_steps

[{'agent': 'Movies and Series Agent Specialist',
  'input': '{\'input\': "I want to write a movie for kids about the chicken \'Jiji\'.  This is the summary:\\n\\nJiji is sad due to her main dream is to be a super hero and defend her siblings against the bad foxes that comes at night in the farm.\\nHowever, she thinks chickens will never become a super hero.\\nThe movie may show the transition of \'Jiji\' from a sad chicken to a super-hero one.\\n\\nCan you please write this script and depict it to me?", \'chat_history\': [], \'outputFormat\': \'text\'}',
  'output': "Here is the script for your movie 'Jiji the Super Chicken':\n\n**Title: Jiji the Super Chicken**\n\n**FADE IN:**\n\n**EXT. FARM - SUNSET**\n\n*The sun sets over a peaceful farm. The camera pans down to reveal a small chicken coop. Inside, we see a sad-looking chicken, JIJI, sitting alone in a corner, her feathers ruffled.*\n\n**NARRATOR (V.O.)**  \nIn a quiet farm, there lived a little chicken named Jiji. While her sibling

## Utility Tools

Is there an utlity missing in our platform that you need to build your agent? No problem! You can onboard it!

Let's onboard the search API of [The Movie Database](https://www.themoviedb.org/)

In [ ]:
def main(start_location: str, end_location: str):
    """
    Get transport directions from start_location to end_location
    """
    import os
    try:
        import googlemaps
    except:
        os.system("pip install googlemaps")
        import googlemaps
    import json
    from datetime import datetime

    # Set up your Google Maps API key
    API_KEY = 'YOUR GOOGLE MAPS KEY HERE'  # Replace with your actual API key

    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=API_KEY)

    # Request directions
    now = datetime.now()
    directions_result = gmaps.directions(
        start_location,
        end_location,
        # mode="transit",  # Use "transit" for public transport
        mode="driving",  # Use "transit" for public transport
        departure_time=now
    )

    try:
        directions = []
        # Output the directions
        for step in directions_result[0]['legs'][0]['steps']:
            directions.append({
                "instructions": step['html_instructions'],
                "duration": step['duration']['text'],
                "distance": step['distance']['text']
            })
        directions = json.dumps(directions)
    except:
        directions = "No directions found. Please check your input."
    return directions

In [ ]:
from aixplain.enums import DataType
from aixplain.factories import ModelFactory
from aixplain.modules.model.utility_model import UtilityModelInput

utility_model = ModelFactory.create_utility_model(
    name="Google Maps instructions",
    code=main,
    output_examples="Driving example"
)

In [ ]:
google_maps_tool = AgentFactory.create_model_tool(model=utility_model.id)

In [ ]:
google_places_tool = AgentFactory.create_model_tool(model="66f6ac496eb563510e3503d1")

In [ ]:
agent.tools.extend([
    google_maps_tool,
    google_places_tool
])
agent.save()

In [ ]:
response = agent.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville?")

In [ ]:
print(response.data.output)

There are several cinemas near Joinville, including: 
1. GNC Cinemas - Shopping Garten, located at Av. Rolf Wiest, 333 - Bom Retiro, Joinville - SC, 89223-005, Brazil. 
2. Cine Uniplex Shopping Cidade das Flores - Joinville, located at R. Mário Lobo, 106 - Centro, Joinville - SC, 89201-330, Brazil. 
3. GNC Cinemas - Shopping Mueller Joinville, located at Shopping Mueller Joinville, R. Sen. Felipe Schmidt, s/n - Centro, Joinville - SC, 89201-420, Brazil. 

To get to GNC Cinemas - Shopping Garten from the Art Museum of Joinville, follow these directions: 
1. Head north on Av. Juscelino Kubitscheck toward Av. Juscelino Kubitscheck (0.4 km, 1 min). 
2. Continue onto R. Dr. João Colin (3.2 km, 7 mins). 
3. Continue straight onto Av. Santos Dumont (55 m, 1 min). 
4. Slight right to stay on Av. Santos Dumont (0.5 km, 1 min). 
5. At the roundabout, take the 1st exit onto R. Dom Bosco (0.6 km, 1 min). 
6. Turn left before MAGRÃO AGROPET (1.6 km, 3 mins). 
7. Turn right onto Av. Santos Dumont (5

In [ ]:
response.data.intermediate_steps[0]["tool_steps"]

[{'tool': 'Places API',
  'input': "{'text': 'cinemas near Joinville'}",
  'output': '[{\'business_status\': \'OPERATIONAL\', \'formatted_address\': \'Av. Rolf Wiest, 333 - Bom Retiro, Joinville - SC, 89223-005, Brazil\', \'geometry\': {\'location\': {\'lat\': -26.2537897, \'lng\': -48.8533947}, \'viewport\': {\'northeast\': {\'lat\': -26.25203157010728, \'lng\': -48.85173915000001}, \'southwest\': {\'lat\': -26.25473122989272, \'lng\': -48.85494894999999}}}, \'icon\': \'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/movies-71.png\', \'icon_background_color\': \'#13B5C7\', \'icon_mask_base_uri\': \'https://maps.gstatic.com/mapfiles/place_api/icons/v2/movie_pinlet\', \'name\': \'GNC Cinemas - Shopping Garten\', \'opening_hours\': {\'open_now\': True}, \'photos\': [{\'height\': 2988, \'html_attributions\': [\'<a href="https://maps.google.com/maps/contrib/108958634086475088109">Alexandre Maros</a>\'], \'photo_reference\': \'AeeoHcK9s5YzQafGrZaZTfDpWXfjykzRtr9F1SHwYbUGgsPFhkIp

## Deploying an Agent

Once created, Agents are saved as draft and are activated for 24 hours. If you are satisfied with the one you developed and want to use longer, deploy it!

In [ ]:
agent.deploy()

## Getting an Agent

You don't have to create an agent everytime you want to use it! Once saved, an agent in aiXplain has an asset ID and you want instantiate it in your Python application with one line of code!

In [ ]:
agent.id

In [ ]:
agent = AgentFactory.get("67f00ac11fbb3d03954c7e38")

In [ ]:
response = agent.run("How many Oscars Angelina Jolie have?")
response.data.output

## Deleting the Agent

Do not want to use your agent anymore? You may delete it then.

In [ ]:
agent.delete()

# Introduction to Teams

## Limitation of Agents

They struggle to reason among multiple tools or to solve queries which require multiple steps to be solved

In [ ]:
INSTRUCTION = "You are a Movies and Series Agent Specialist. Your role is to answer user queries about movies and series in a fluent and assertive manner. For questions that are not related to movies and series, indicate that you are not a specialist in that area and cannot provide answers."

DESCRIPTION = "A Movies and Series Agent Specialist"

agent = AgentFactory.create(
    name="Movies and Series Agent Specialist",
    tools=[
        speech_synthesis_tool,
        asr_tool,
        translation_tool,
        search_tool,
        pipeline_tool,
        google_places_tool,
        google_maps_tool
    ],
    description=DESCRIPTION,
    instructions=INSTRUCTION,
)

In [ ]:
response = agent.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?")

In [ ]:
print(response.data.output)

Here are some cinemas near Joinville: 1. GNC Cinemas - Shopping Garten, located at Av. Rolf Wiest, 333 - Bom Retiro, Joinville - SC. 2. Cine Uniplex Shopping Cidade das Flores - Joinville, located at R. Mário Lobo, 106 - Centro, Joinville - SC. 3. GNC Cinemas - Shopping Mueller Joinville, located at Shopping Mueller Joinville, R. Sen. Felipe Schmidt, s/n - Centro, Joinville - SC. 4. Cine Uniplex Shopping Estação - Joinville, located at Av. Getúlio Vargas, 1446 - Bucarein, Joinville - SC. 5. UAU FILMES, located at R. Xavantes, 54 - Atiradores, Joinville - SC. 6. Jocemar Maçaneiro, located at R. Graciosa, 1132 - Guanabara, Joinville - SC. To get to one of these cinemas from the Art Museum of Joinville, you can use a navigation app for directions. As for what to watch tonight, please check the cinema listings for current showtimes and available movies.


## Mentalist

For such scenarios you may consider our team approach. In teams we may enable mentalist to predict all the steps that should be solved ahead. The agent will be then called by orchestrator to solve each step of the plan and will not be overloaded.

In [ ]:
from aixplain.factories import TeamAgentFactory

team = TeamAgentFactory.create(
    name="Movies and Series Agent Specialist",
    agents=[agent],
    use_mentalist=True,
    use_inspector=False
)

In [ ]:
response = team.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?")

In [ ]:
print(response["data"]["output"])

Here’s the information you requested:

### Cinemas Near Joinville
In Joinville, you can find several cinemas, including:
1. **Cinepolis Joinville** - Located in the Shopping Mueller.
2. **Cinemark Joinville** - Situated in the Shopping Garten.
3. **Cineflix Joinville** - Found in the Shopping Cidade das Flores.

### Getting to a Cinema from the Art Museum of Joinville
To reach one of these cinemas from the Art Museum of Joinville, you can take a taxi or use a rideshare service for convenience. Alternatively, if you prefer public transport, check local bus routes that connect the museum to the shopping centers where the cinemas are located.

### Movie Showtimes for Tonight
For tonight's movie options, I recommend checking the official websites or apps of the cinemas mentioned above. They typically provide up-to-date listings of movies, showtimes, and ticket availability.

If you have any more questions or need further assistance, feel free to ask!


In [ ]:
response["data"]["intermediate_steps"][0]

{'agent': 'mentalist',
 'input': 'INPUT\n--------------------\n\nUSER: What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?',
 'output': "[{'step': 'Find cinemas near Joinville', 'worker': 'Places API', 'init_worker': 'Places API'}, {'step': 'Get directions from Art Museum of Joinville to one of the cinemas', 'worker': 'Google Maps instructions', 'init_worker': 'Google Maps instructions'}, {'step': 'Provide movie options for tonight', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}]",
 'tool_steps': None,
 'thought': 'The user is asking for information about cinemas near Joinville, directions from the Art Museum of Joinville to one of those cinemas, and movie options for tonight. The first two tasks can be handled by the Places API and Google Maps instructions, while the last task requires the Movies and Series Agent Specialist to provide movie options.',
 'runTime': 2.

## Better Team Architecture

The previous outcome is not good yet. We could change the LLM from GPT-4o Mini to a more powerful one. But let's keep our agent cheap and just better architecture our team, spliting the expertise among two specialists.

In [ ]:
INSTRUCTION = "You are a Movies and Series Agent Specialist. Your role is to answer user queries about movies and series in a fluent and assertive manner. For questions that are not related to movies and series, indicate that you are not a specialist in that area and cannot provide answers."
DESCRIPTION = "A Movies and Series Agent Specialist"

movie_expert = AgentFactory.create(
    name="Movies and Series Agent Specialist",
    tools=[
        speech_synthesis_tool,
        asr_tool,
        translation_tool,
        search_tool,
        pipeline_tool,
    ],
    description=DESCRIPTION,
    instructions=INSTRUCTION,
)

In [ ]:
google_places_tool,
google_maps_tool
INSTRUCTION = "You are a Cinema Finder. Your role is to answer user queries about cinemas and what to what on them in a fluent and assertive manner. For questions that are not related, indicate that you are not a specialist in that area and cannot provide answers."
DESCRIPTION = "Find the best cinemas near you"

cinema_finder = AgentFactory.create(
    name="Cinema Finder",
    tools=[
        google_places_tool,
        google_maps_tool,
        search_tool,
    ],
    description=DESCRIPTION,
    instructions=INSTRUCTION,
)

In [ ]:
from aixplain.factories import TeamAgentFactory

team = TeamAgentFactory.create(
    name="Movies and Series Agent Specialist",
    agents=[
        movie_expert,
        cinema_finder
    ],
    use_mentalist=True,
    use_inspector=False
)

In [ ]:
response = team.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?")
print(response["data"]["output"])

"Here’s the information you requested regarding cinemas near Joinville, directions from the Art Museum of Joinville, and what you can watch tonight:\n\n### Cinemas Near Joinville\n1. **GNC Cinemas - Shopping Garten**  \n   Address: Av. Rolf Wiest, 333 - Bom Retiro, Joinville - SC, 89223-005, Brazil  \n   Rating: 4.6 (5153 ratings)  \n   Open Now: Yes  \n\n2. **Cine Uniplex Shopping Cidade das Flores - Joinville**  \n   Address: R. Mário Lobo, 106 - Centro, Joinville - SC, 89201-330, Brazil  \n   Rating: 4.6 (4183 ratings)  \n   Open Now: Yes  \n\n3. **GNC Cinemas - Shopping Mueller Joinville**  \n   Address: Shopping Mueller Joinville, R. Sen. Felipe Schmidt, s/n - Centro, Joinville - SC, 89201-420, Brazil  \n   Rating: 4.5 (5670 ratings)  \n   Open Now: Yes  \n\n4. **Cine Uniplex Shopping Estação - Joinville**  \n   Address: Av. Getúlio Vargas, 1446 - Bucarein, Joinville - SC, 89202-002, Brazil  \n   Rating: 4.5 (24 ratings)  \n   Open Now: No  \n\n5. **UAU FILMES**  \n   Address: R. 

In [ ]:
response["data"]["intermediate_steps"][0]

{'agent': 'mentalist',
 'input': 'INPUT\n--------------------\n\nUSER: What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?',
 'output': "[{'step': 'Find cinemas near Joinville', 'worker': 'Cinema Finder', 'init_worker': 'Cinema Finder'}, {'step': 'Get directions from Art Museum of Joinville to one of the cinemas', 'worker': 'Cinema Finder', 'init_worker': 'Cinema Finder'}, {'step': 'Find out what movies are showing tonight', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}]",
 'tool_steps': None,
 'thought': 'The user is asking for information about cinemas near Joinville, directions from the Art Museum of Joinville to one of those cinemas, and movie options for tonight. Each part of the query can be addressed by the appropriate workers.',
 'runTime': 2.633,
 'usedCredits': 0.00020879999999999998,
 'apiCalls': 1,
 'task': None}

## Tasks

If you prefer, enable the steps of the plan yourself instead of mentalist

In [ ]:
task1 = AgentFactory.create_task(
    name="Cinema Search",
    description="Search for cinemas near the area specified by the user",
    expected_output="A list of cinemas near the specified area",
)

task2 = AgentFactory.create_task(
    name="Movie Search",
    description="Search for movies near the area specified by the user",
    expected_output="A list of movies near the specified area",
)

task3 = AgentFactory.create_task(
    name="Find Directions",
    description="Find the directions between the cinema address and the user address",
    expected_output="A detailed driving direction from the user address to the cinema one",
)

In [ ]:
agent.tasks.extend([
    task1, task2, task3
])
agent.save()

/usr/local/lib/python3.11/dist-packages/aixplain/modules/model/utility_model.py:173: UserWarning: WARNING: Non-deployed utility models (status=DRAFT) will expire after 24 hours after creation. Use .deploy() method to make the model permanent.
  warnings.warn(


In [ ]:
from aixplain.factories import TeamAgentFactory

team = TeamAgentFactory.create(
    name="Movies and Series Agent Specialist",
    agents=[agent],
    use_mentalist=False,
    use_inspector=False
)

In [ ]:
response = team.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch there tonight?")

In [ ]:
print(response["data"]["output"])

Here’s the information you requested regarding cinemas near Joinville, directions from the Art Museum of Joinville, and movie options for tonight:

### Cinemas Near Joinville
1. **GNC Cinemas - Shopping Garten**
   - Address: Av. Rolf Wiest, 333 - Bom Retiro, Joinville - SC, 89223-005, Brazil
   - Rating: 4.6

2. **Cine Uniplex Shopping Cidade das Flores - Joinville**
   - Address: R. Mário Lobo, 106 - Centro, Joinville - SC, 89201-330, Brazil
   - Rating: 4.6

3. **GNC Cinemas - Shopping Mueller Joinville**
   - Address: Shopping Mueller Joinville, R. Sen. Felipe Schmidt, s/n - Centro, Joinville - SC, 89201-420, Brazil
   - Rating: 4.5

4. **Cine Uniplex Shopping Estação - Joinville**
   - Address: Av. Getúlio Vargas, 1446 - Bucarein, Joinville - SC, 89202-002, Brazil
   - Rating: 4.5

5. **UAU FILMES**
   - Address: R. Xavantes, 54 - Atiradores, Joinville - SC, 89203-900, Brazil
   - Rating: 4.8

6. **Jocemar Maçaneiro**
   - Address: R. Graciosa, 1132 - Guanabara, Joinville - SC, 89

In [ ]:
response["data"]["intermediate_steps"][0]

{'agent': 'mentalist',
 'input': '',
 'output': "[{'step': 'Task: Cinema Search\\n\\t\\tDescription: Search for cinemas near the area specified by the user\\n\\t\\tExpected output: A list of cinemas near the specified area', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}, {'step': 'Task: Movie Search\\n\\t\\tDescription: Search for movies near the area specified by the user\\n\\t\\tExpected output: A list of movies near the specified area', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}, {'step': 'Task: Find Directions\\n\\t\\tDescription: Find the directions between the cinema address and the user address\\n\\t\\tExpected output: A detailed driving direction from the user address to the cinema one', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}]",
 'tool_steps': None,
 'thought': 'Planning steps provided by user',
 'runTime'

## Inspector

Do you want the outcomes of the agent to be refined by an LLM as a judge? Use Inspector!

In [ ]:
team.delete()

In [ ]:
from aixplain.factories import TeamAgentFactory

# team.delete()
team = TeamAgentFactory.create(
    name="Movies and Series Agent Specialist 1",
    agents=[agent],
    use_mentalist=True,
    use_inspector=True
)

In [ ]:
response = team.run("What are cinemas near Joinville? How can I get in one of them from Art Museum of Joinville? What can I watch tonight?")

In [ ]:
print(response["data"]["output"])

Here’s the information you requested regarding cinemas near Joinville, directions from the Art Museum of Joinville, and what you can watch tonight:

### Cinemas Near Joinville:
1. **GNC Garten**
   - **Address:** Avenida Rolf Wiest 333, Joinville
   - **Movies Showing:**
     - Branca De Neve (Adventure, Fantasy) - Showtime: 19:00
     - Drop: Ameaça Anônima (Action, Drama, Thriller) - Showtime: 19:00
     - Nas Terras Perdidas (Action, Fantasy) - Showtime: 19:00

2. **GNC Joinville Mueller**
   - **Movies Showing:**
     - Operação Vingança (Suspense) - Various showtimes available today.

3. **Cinemas Uniplex Joinville Shopping**
   - **Movies Showing:**
     - The Chosen - Última Ceia (Drama) - Showtimes available today.

4. **Cinemas Uniplex - Joinville Estação**
   - **Showtimes available today.**

### Directions:
Please let me know which cinema you would like directions to: 
- GNC Garten
- GNC Joinville Mueller
- Cinemas Uniplex Joinville Shopping
- Cinemas Uniplex - Joinville Est

In [ ]:
for step in response["data"]["intermediate_steps"]:
  print(step)

{'agent': 'mentalist', 'input': '', 'output': "[{'step': 'Task: Cinema Search\\n\\t\\tDescription: Search for cinemas near the area specified by the user\\n\\t\\tExpected output: A list of cinemas near the specified area', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}, {'step': 'Task: Movie Search\\n\\t\\tDescription: Search for movies near the area specified by the user\\n\\t\\tExpected output: A list of movies near the specified area', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}, {'step': 'Task: Find Directions\\n\\t\\tDescription: Find the directions between the cinema address and the user address\\n\\t\\tExpected output: A detailed driving direction from the user address to the cinema one', 'worker': 'Movies and Series Agent Specialist', 'init_worker': 'Movies and Series Agent Specialist'}]", 'tool_steps': None, 'thought': 'Planning steps provided by user', 'runTime': 0, 